# Домашнее задание 5. Линейные модели

In [17]:
import random as pr
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import sklearn.cross_validation as cv
import sklearn.metrics as sm

# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


Зачитываем результат 4 домашки

In [2]:
data = np.load("files/out_4.dat.npz")
users = data["users"]
X = data["data"].reshape(1,)[0]

Зачитываем категории пользователей

In [3]:
TRAINING_SET_URL = "twitter_train.csv"
df_users = pd.read_csv(TRAINING_SET_URL, sep=",", header=0, names=["twitter_id", "is_1", "is_2", "is_3"], dtype={"twitter_id": str, "is_1": int, 'is_2': int, "is_3": int})
df_users.set_index("twitter_id", inplace=True)

Формируем целевую переменную: Делаем join списка пользователей из ДЗ4 с обучающей выборкой.

In [4]:
Y_1 = df_users.ix[users.astype(str)]["is_1"].values
Y_2 = df_users.ix[users.astype(str)]["is_2"].values
Y_3 = df_users.ix[users.astype(str)]["is_3"].values
print "Resulting training set for 1 class: (%dx%d) feature matrix, %d target vector" % (X.shape[0], X.shape[1], Y_1.shape[0])
print "Resulting training set for 2 class: (%dx%d) feature matrix, %d target vector" % (X.shape[0], X.shape[1], Y_2.shape[0])
print "Resulting training set for 3 class: (%dx%d) feature matrix, %d target vector" % (X.shape[0], X.shape[1], Y_3.shape[0])

Resulting training set for 1 class: (3001x70832) feature matrix, 3001 target vector
Resulting training set for 2 class: (3001x70832) feature matrix, 3001 target vector
Resulting training set for 3 class: (3001x70832) feature matrix, 3001 target vector


Чтобы исследовать, как ведут себя признаки, построим распределение количества ненулевых признаков у пользователей, чтобы убедиться, что он удовлетворяет закону Ципфа.

In [7]:
def draw_log_hist(x):
    """Draw tokens histogram in log scales"""
    
    # Your code here
    
    return [0] * x.shape[1]

features_counts = draw_log_hist(X)

Проведем отбор признаков. В самом простом случае просто удаляем признаки, имеющие ненулевое значение у менее, чем 100 пользователей.

In [8]:
X1 = X.tocsc()[:, features_counts > 100].toarray()

Вариант задания генерируется на основании вашего ника в техносфере.

In [9]:
USER_NAME = "n.anokhin"
OPTIMIZATION_ALGORITHMS = ["stochastic gradient descent", "Newton method"]
REGULARIZATIONS = ["L1", "L2"]

print "My homework 5 algorithm is: Logistic regression with %s regularization optimized by %s" % (
    REGULARIZATIONS[hash(USER_NAME) % 2],
    OPTIMIZATION_ALGORITHMS[hash(USER_NAME[::-1]) % 2]
)

My homework 5 algorithm is: Logistic regression with L2 regularization optimized by Newton method


Реализуем выбранный алгоритм

In [10]:
class LogisticRegression():
    
    def fit(self, X, Y=None):
        return self
    
    def predict_proba(self, X):
        import numpy.random as nr
        return nr.random((X.shape[0], 2))

Реализуем метрику качества, используемую в соревновании: площадь под ROC кривой

In [11]:
def auroc(y_prob, y_true):
    return 0.5

Разделим выборку с помощью методики кросс-валидации для того, чтобы настроить параметр регуляризации $C$

In [14]:
C = [0.0, 0.01, 0.1, 1, 10, 100, 1000, 10000]

def select_reg_parameter(C, X, Y):
    
    return C.index(max(C))

index1 = select_reg_parameter(C, X1, Y_1)
print index1

index2 = select_reg_parameter(C, X1, Y_2)
print index2

index3 = select_reg_parameter(C, X1, Y_3)
print index3

7
7
7


Выбираем наилучшее значение $C$, и классифицируем неизвестных пользователей и строим ROC-кривую

In [16]:
def classify(X, Y, test_size, C):
    tpr = [1] * 2400
    fpr = [0.01] * 2400
    roc_auc = 0.51
    
    return tpr, fpr, roc_auc

def plot_roc_curve(tpr, fpr, roc_auc):
    """Plot ROC curve"""
    
    # Your code here
    
    return

tpr, fpr, roc_auc = classify(X1, Y_1, 0.3, C[index1])

print "Area under the ROC curve : %f" % roc_auc

plot_roc_curve(tpr, fpr, roc_auc)

tpr, fpr, roc_auc = classify(X1, Y_2, 0.3, C[index2])

print "Area under the ROC curve : %f" % roc_auc

plot_roc_curve(tpr, fpr, roc_auc)

tpr, fpr, roc_auc = classify(X1, Y_3, 0.3, C[index3])

print "Area under the ROC curve : %f" % roc_auc

plot_roc_curve(tpr, fpr, roc_auc)

Area under the ROC curve : 0.510000
Area under the ROC curve : 0.510000
Area under the ROC curve : 0.510000


С помощью полученной модели предсказываем категории для неизвестных пользователей из соревнования и загружаем на kaggle в нужном формате.